## Задача 3-2. Задача TSP: нижняя оценка Гельда—Карпа.

В этой задаче Вам предлагается релизовать алгоритм Гельда—Карпа для нижней оценки стоимости решения в задаче Euclidean TSP.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP. Это в точности те же входные данные, что и в задании 3-1.
* Реализуйте функцию `lower_bound_tsp`. При этом можно пользоваться каким-нибудь стандартным алгоритмом построения минимального остовного дерева из библиотеки [`networkx`](https://networkx.github.io/), входящей в состав дистрибутива Anaconda.
* Запустите функцию `run_all()`, чтобы протестировать свой код, и напишите полученные, как следствия, верхние оценки погрешностей решений, которые были получены Вашими алгоритмами NN и NI при решении задания 3-1. Запишите свои выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [1]:
from typing import List, Tuple
from math import sqrt, inf
from itertools import combinations, islice

import networkx as nx

Point = Tuple[int, int]

In [2]:
def read_tsp_instance(filename: str) -> List[Point]:
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0, 0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates


def euclidean_distance(point1: Point, point2: Point) -> float:
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)

In [19]:
def full_euclidean_graph(vertex_coordinates: List[Point]):
    graph = nx.Graph()
    
    for v in vertex_coordinates:
        graph.add_node(v)
    
    for v1, v2 in combinations(vertex_coordinates, 2):
        graph.add_edge(v1, v2, weight=euclidean_distance(v1, v2))
    
    return graph

def graph_weight(g: nx.Graph) -> float:
    return sum([edge['weight'] for edge in g.edges(data=True)])

def tree_is_route(g: nx.Graph) -> bool:
    degrees = nx.degree(g)
    n = g.number_of_nodes()
    return all([degrees[v] <= 2 for v in degrees]) and (sum(degrees.values()) == (2 * (n - 2) + 1 + 1))


In [97]:
def lower_bound_tsp(vertex_coordinates: List[Point]) -> float:
    g = full_euclidean_graph(vertex_coordinates)
    
    some_traversal = list(g.nodes())
    some_traversal.append(some_traversal[0])
    
    # some upper bound for tour length
    U = sum(g[some_traversal[i]][some_traversal[i + 1]]['weight'] for i in range(len(some_traversal) - 1))
    
    y = {v: 0 for v in g.nodes()}
    best_held_carp_bound = -float('inf')
    t_lowest = 0.0001
    alpha = 2
    beta = 2 / 3
    iterations_factor = 0.015
    
    max_changes = 100
    num_iterations = int(g.number_of_nodes() * iterations_factor)
    
    for i in range(max_changes):
        for k in range(num_iterations):
            for v1, v2 in combinations(g.nodes(), 2):
                g[v1][v2]['weight'] = g[v1][v2]['weight'] - y[v1] - y[v2]
            
            optimal = False
            v1 = g.nodes()[0]
            v1_edges = g[v1]
            g.remove_node(v1)
            rest_mst = nx.minimum_spanning_tree(g)
            
            g.add_node(v1)
            for v1_incident in v1_edges:
                g.add_edge(v1, v1_incident, weight=v1_edges[v1_incident]['weight'])
            
            rest_mst.add_node(v1)
            sorted_by_distance = sorted(rest_mst.nodes(), key=lambda v: euclidean_distance(v, v1))
            v_e = sorted_by_distance[1]
            v_f = sorted_by_distance[2]

            if rest_mst.degree(v_e) == 1 and rest_mst.degree(v_f) == 1:
                optimal = True

            rest_mst.add_edge(v1, v_e, weight=g[v1][v_e]['weight'])
            rest_mst.add_edge(v1, v_f, weight=g[v1][v_f]['weight'])
            
            assert rest_mst.number_of_edges() == g.number_of_nodes()
            
            held_carp_bound = sum([edge[2]['weight'] for edge in rest_mst.edges(data=True)])
            
            best_held_carp_bound = max(best_held_carp_bound, held_carp_bound)
            if optimal:
                print ("ОПТИМАЛЬНЕНЬКО")
                return best_held_carp_bound + 2 * sum((y[v] for v in g.nodes()))
            
            t_cur = alpha*(U-held_carp_bound) / sum([(2-rest_mst.degree(v))**2 for v in rest_mst.nodes()])
            if t_cur < t_lowest:
                return best_held_carp_bound + 2 * sum((y[v] for v in g.nodes()))
            
            for v1, v2 in combinations(g.nodes(), 2):
                g[v1][v2]['weight'] = g[v1][v2]['weight'] + y[v1] + y[v2]
                        
            for v in g.nodes():
                y[v] = y[v] + t_cur * (2 - rest_mst.degree(v))
            
            alpha = alpha * beta
    
    return best_held_carp_bound + 2 * sum((y[v] for v in g.nodes()))

In [98]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Instance {}…'.format(filename), end='\n')
        time_start = time.monotonic()
        bound = lower_bound_tsp(instance)
        time_nn = time.monotonic()-time_start
        print(' done in {:.2} seconds with lower bound {}'.format(time_nn, int(bound)))

In [99]:
run_all()

Instance d198.tsp…
ОПТИМАЛЬНЕНЬКО
 done in 0.36 seconds with lower bound 11831
Instance d493.tsp…
ОПТИМАЛЬНЕНЬКО
 done in 2.0 seconds with lower bound 29406
Instance d657.tsp…
ОПТИМАЛЬНЕНЬКО
 done in 3.7 seconds with lower bound 42589
Instance d2103.tsp…
ОПТИМАЛЬНЕНЬКО
 done in 2.4e+01 seconds with lower bound 76351
Instance pr107.tsp…
ОПТИМАЛЬНЕНЬКО
 done in 0.076 seconds with lower bound 35090
Instance pr152.tsp…
ОПТИМАЛЬНЕНЬКО
 done in 0.15 seconds with lower bound 59528
Instance pr439.tsp…
ОПТИМАЛЬНЕНЬКО
 done in 1.4 seconds with lower bound 92429


## Выводы

Получили

| Тип                          	| d198  	| d493  	| d657  	| d2103 	| pr107 	| pr152 	| pr439  	|
|------------------------------	|-------	|-------	|-------	|-------	|-------	|-------	|--------	|
| HK                           	| 11831 	| 29406 	| 42589 	| 76351 	| 35090 	| 59528 	| 92429  	|
| NI                           	| 17631 	| 39982 	| 57906 	| 87570 	| 52587 	| 88530 	| 130067 	|
| Абсолютная погрешность NI    	| 49.02 	| 35.97 	| 35.96 	| 14.69 	| 49.86 	| 48.72 	| 40.72  	|
| Относительная погрешность NI 	| 1.49  	| 1.36  	| 1.36  	| 1.15  	| 1.5   	| 1.49  	| 1.41   	|
| NN                           	| 18620 	| 43646 	| 62176 	| 87468 	| 46678 	| 85702 	| 131282 	|
| Абсолютная погрешность NN    	| 57.38 	| 48.43 	| 45.99 	| 14.56 	| 33.02 	| 43.97 	| 42.04  	|
| Относительная погрешность NN 	| 1.57  	| 1.48  	| 1.46  	| 1.15  	| 1.33  	| 1.44  	| 1.42   	|